In [ ]:
#do not use...needs full correlation matrix as input 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load features and targets from two different files
features_file = r"C:\Users\Administrator\Desktop\code\version 2\sentinel 2\sentinel_bandratios_ratios.csv"
targets_file = r"C:\Users\Administrator\Desktop\code\version 2\sentinel 2\sparameters.csv"
correlation_file = r"C:\Users\Administrator\Desktop\code\version1\temp_spara.xlsx"

df_features = pd.read_csv(features_file)
df_targets = pd.read_csv(targets_file)
correlation_matrix = pd.read_excel(correlation_file)
correlation_matrix = correlation_matrix.loc[:, ~correlation_matrix.columns.str.contains('^Unnamed')]

# Clean column names
df_features.columns = df_features.columns.str.strip()
df_targets.columns = df_targets.columns.str.strip()

# Function to perform regression and metric evaluation
def evaluate_correlated_pairs(corr_matrix, df_features, df_targets, threshold=0.3, train_size=0.7):
    results = []

    for feature in df_features.columns:
        for target in df_targets.columns:
            try:
                corr_value = corr_matrix.loc[feature, target]
            except KeyError:
                continue  # Skip if the correlation matrix doesn't include this pair

            if abs(corr_value) >= threshold:
                X = df_features[[feature]].values.reshape(-1, 1)
                y = df_targets[target].values

                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42)

                # Fit linear model
                model = LinearRegression()
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                # Metrics
                r_squared = model.score(X_test, y_test)
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                mae = mean_absolute_error(y_test, y_pred)
                mean_obs = np.mean(y_test)
                nse = 1 - np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2)

                results.append({
                    'Feature': feature,
                    'Target': target,
                    'Correlation': corr_value,
                    'Slope': model.coef_[0],
                    'Intercept': model.intercept_,
                    'R_squared': r_squared,
                    'RMSE': rmse,
                    'MAE': mae,
                    'NSE': nse
                })

                print(f"{feature} → {target}: R²={r_squared:.3f}, RMSE={rmse:.3f}, MAE={mae:.3f}, NSE={nse:.3f}")

    return pd.DataFrame(results)

# Run evaluation
final_results = evaluate_correlated_pairs(correlation_matrix, df_features, df_targets, threshold=0.3)

# Save to a file if desired
output_path = r"C:\Users\Administrator\Desktop\correlation_regression_results.parquet"
final_results.to_parquet(output_path, index=False)
print(f"\nResults saved to {output_path}")


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\dataforcode\sent2(GYr).xlsx"
targets_file = r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\sbp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_excel(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")



Processing block: 0 to 10

Results saved to: C:\Users\Administrator\Desktop\code\band to parameter\sbp_lr.parquet


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sentinel_bandratios_ratios.csv"
targets_file = r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\srp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_csv(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")



Processing block: 0 to 90

Results saved to: C:\Users\Administrator\Desktop\code\band to parameter\srp_lr.parquet


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sentinel_linearcombinations4_evaluated.csv"
targets_file = r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\slcp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_csv(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")


Processing block: 0 to 100

Processing block: 100 to 200

Processing block: 200 to 300

Processing block: 300 to 400

Processing block: 400 to 500

Processing block: 500 to 600

Processing block: 600 to 700

Processing block: 700 to 800

Processing block: 800 to 900

Processing block: 900 to 1000

Processing block: 1000 to 1100

Processing block: 1100 to 1200

Processing block: 1200 to 1300

Processing block: 1300 to 1400

Processing block: 1400 to 1500

Processing block: 1500 to 1600

Processing block: 1600 to 1700

Processing block: 1700 to 1800

Processing block: 1800 to 1900

Processing block: 1900 to 2000

Processing block: 2000 to 2100

Processing block: 2100 to 2200

Processing block: 2200 to 2300

Processing block: 2300 to 2400

Processing block: 2400 to 2500

Processing block: 2500 to 2600

Processing block: 2600 to 2700

Processing block: 2700 to 2800

Processing block: 2800 to 2900

Processing block: 2900 to 3000

Processing block: 3000 to 3100

Processing block: 3100 to 32

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sentinel_2bandcombratios_ratios.csv"
targets_file = r"C:\Users\Administrator\Desktop\code\band to band\sentinel 2\sparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\s2lrp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_csv(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")


Processing block: 0 to 100

Processing block: 100 to 200

Processing block: 200 to 300

Processing block: 300 to 400

Processing block: 400 to 500

Processing block: 500 to 600

Processing block: 600 to 700

Processing block: 700 to 800

Processing block: 800 to 900

Processing block: 900 to 1000

Processing block: 1000 to 1100

Processing block: 1100 to 1200

Processing block: 1200 to 1300

Processing block: 1300 to 1400

Processing block: 1400 to 1500

Processing block: 1500 to 1600

Processing block: 1600 to 1700

Processing block: 1700 to 1800


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Processing block: 1800 to 1900


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Processing block: 1900 to 2000

Processing block: 2000 to 2100


...


Processing block: 28400 to 28500

Processing block: 28500 to 28600


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Processing block: 28600 to 28700

Processing block: 28700 to 28731


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Results saved to: C:\Users\Administrator\Desktop\code\band to parameter\s2lrp_lr.parquet


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\dataforcode\LS8,9(GYr).xlsx"
targets_file = r"C:\Users\Administrator\Desktop\code\version 2\landsat8,9\lparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\lbp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_excel(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")


Processing block: 0 to 8

Results saved to: C:\Users\Administrator\Desktop\code\band to parameter\sbp_lr.parquet


In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\code\version 2\landsat8,9\landsat_bandratios.csv"
targets_file = r"C:\Users\Administrator\Desktop\code\version 2\landsat8,9\lparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\lrp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_csv(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")


Processing block: 0 to 56

Results saved to: C:\Users\Administrator\Desktop\code\band to parameter\lrp_lr.parquet


In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\code\version 2\landsat8,9\landsat_linearcombinations4.csv"
targets_file = r"C:\Users\Administrator\Desktop\code\version 2\landsat8,9\lparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\llcp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_csv(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")


Processing block: 0 to 100

Processing block: 100 to 200

Processing block: 200 to 300

Processing block: 300 to 400

Processing block: 400 to 500

Processing block: 500 to 600

Processing block: 600 to 700

Processing block: 700 to 800

Processing block: 800 to 900

Processing block: 900 to 1000

Processing block: 1000 to 1100

Processing block: 1100 to 1200

Processing block: 1200 to 1300

Processing block: 1300 to 1400

Processing block: 1400 to 1500

Processing block: 1500 to 1600

Processing block: 1600 to 1688

Results saved to: C:\Users\Administrator\Desktop\code\band to parameter\llcp_lr.parquet


In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# File paths
features_file = r"C:\Users\Administrator\Desktop\code\version 2\landsat8,9\landsat_2bandcombratios_ratios.csv"
targets_file = r"C:\Users\Administrator\Desktop\code\version 2\landsat8,9\lparameters.csv"
output_path = r"C:\Users\Administrator\Desktop\code\band to parameter\l2lrp_lr.parquet"

# Load full target dataset (assuming it's smaller or manageable)
df_targets = pd.read_csv(targets_file)
df_targets.columns = df_targets.columns.str.strip()

# Load feature dataset in chunks (e.g., block of 50 columns)
df_features_all = pd.read_csv(features_file)
df_features_all.columns = df_features_all.columns.str.strip()

block_size = 100
threshold = 0.3
results = []

for start in range(0, df_features_all.shape[1], block_size):
    end = min(start + block_size, df_features_all.shape[1])
    df_features = df_features_all.iloc[:, start:end]
    print(f"\nProcessing block: {start} to {end}")

    for feature in df_features.columns:
        for target in df_targets.columns:
            # Drop rows with NaNs for current pair
            data = pd.concat([df_features[feature], df_targets[target]], axis=1).dropna()
            if data.shape[0] < 10:
                continue

            X = data[[feature]].values
            y = data[target].values

            # Check Pearson correlation
            corr = np.corrcoef(data[feature], data[target])[0, 1]
            if np.abs(corr) < threshold:
                continue

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

            # Fit model
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            r2 = model.score(X_test, y_test)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            mae = mean_absolute_error(y_test, y_pred)
            mean_obs = np.mean(y_test)
            nse = 1 - (np.sum((y_test - y_pred) ** 2) / np.sum((y_test - mean_obs) ** 2))

            # Store result
            results.append({
                'Feature': feature,
                'Target': target,
                'Correlation': corr,
                'Slope': model.coef_[0],
                'Intercept': model.intercept_,
                'R_squared': r2,
                'RMSE': rmse,
                'MAE': mae,
                'NSE': nse
            })


# Save final results
results_df = pd.DataFrame(results)
results_df.to_parquet(output_path, index=False)
print(f"\nResults saved to: {output_path}")


Processing block: 0 to 100

Processing block: 100 to 200

Processing block: 200 to 300

Processing block: 300 to 400

Processing block: 400 to 500

Processing block: 500 to 600

Processing block: 600 to 700

Processing block: 700 to 800

Processing block: 800 to 900


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Processing block: 900 to 1000


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Processing block: 1000 to 1100


...

C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Processing block: 10600 to 10700


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


Processing block: 10700 to 10713

Results saved to: C:\Users\Administrator\Desktop\code\band to parameter\l2lrp_lr.parquet


C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in